In [1]:
import $exclude.`org.slf4j:slf4j-log4j12`, $ivy.`org.slf4j:slf4j-nop:1.7.21` // for cleaner logs
import $profile.`hadoop-2.7`
import $ivy.`org.apache.spark::spark-sql:2.3.0` // adjust spark version - spark >= 2.0
import $ivy.`org.jupyter-scala::spark:0.4.2` // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)
import $ivy.`org.postgresql:postgresql:42.2.2`


import org.apache.spark._
import org.apache.spark.sql._
import jupyter.spark.session._

import org.apache.spark.sql.SparkSession

val spark = SparkSession
      .builder
      .master("local")
      .appName("Test")
      .config("spark.jars", "/mnt/data/dev/workspaces/poc/bigdata-poc/lib/postgresql-42.2.2.jar")
      .getOrCreate()

val sc = spark.sparkContext

log4j:WARN No appenders could be found for logger (io.netty.util.internal.logging.InternalLoggerFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


import $exclude.$                        , $ivy.$                            // for cleaner logs

import $profile.$           

import $ivy.$                                   // adjust spark version - spark >= 2.0

import $ivy.$                                // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)

import $ivy.$                                 



import org.apache.spark._

import org.apache.spark.sql._

import jupyter.spark.session._


import org.apache.spark.sql.SparkSession


spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@14ee7615
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@4a81b901

In [9]:
import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType, Metadata}
import org.apache.spark.sql.functions.{col, column, expr, lit, get_json_object, json_tuple, udf}


import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType, Metadata}

import org.apache.spark.sql.functions.{col, column, expr, lit, get_json_object, json_tuple, udf}


### Json

In [3]:
val jsonDF = spark.range(1).selectExpr("""  '{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}' as jsonString""")

jsonDF: DataFrame = [jsonString: string]

In [4]:
jsonDF.select(
    get_json_object(col("jsonString"), "$.myJSONKey.myJSONValue[1]") as "column",
    json_tuple(col("jsonString"), "myJSONKey")).show(2)

+------+--------------------+
|column|                  c0|
+------+--------------------+
|     2|{"myJSONValue":[1...|
+------+--------------------+



### UDF

In [5]:
def power3(number: Double) : Double = number * number * number


defined function power3

In [6]:
power3(4.0)

res5: Double = 64.0

In [7]:
val udfExampleDF = spark.range(5).toDF("num")

udfExampleDF: DataFrame = [num: bigint]

In [8]:
udfExampleDF

res7: DataFrame = [num: bigint]

In [19]:
//DataFrame function:
val power3udf = udf(power3(_:Double):Double)

power3udf: expressions.UserDefinedFunction = UserDefinedFunction(<function1>, DoubleType, Some(List(DoubleType)))

In [22]:
udfExampleDF.select(power3udf(col("num"))).show()

+--------+
|UDF(num)|
+--------+
|     0.0|
|     1.0|
|     8.0|
|    27.0|
|    64.0|
+--------+



In [23]:
udfExampleDF.select(power3udf(col("num"))).show(2)

+--------+
|UDF(num)|
+--------+
|     0.0|
|     1.0|
+--------+
only showing top 2 rows



In [24]:
//Spark SQL function
spark.udf.register("power3", power3(_:Double):Double)

res23: expressions.UserDefinedFunction = UserDefinedFunction(<function1>, DoubleType, Some(List(DoubleType)))

In [26]:
spark.udf.register("power3", power3(_:Double):Double)
udfExampleDF.selectExpr("power3(num)").show(2)

: 